In [1]:
import gensim, logging
# the model is organized like this: word = embeddings
model = gensim.models.KeyedVectors.load_word2vec_format('../resources/small-embeddings.txt', binary=False)


In [2]:
import nltk, string
from nltk.corpus import stopwords

exclude = set(string.punctuation)
stop_word_list = stopwords.words('english')

# input should be a string
def text_embedding(text):
    
    #it depends if the words have been lowercased or not
    text = text.lower()
    
    text = nltk.word_tokenize(text)
        
    text = [token for token in text if token not in exclude and token.isalpha()]
    
    text = [token for token in text if token not in stop_word_list]

    article_embedd = []
    
    for word in text:
            try:
                embed_word = model[word]
                article_embedd.append(embed_word)
            except KeyError:
                continue

    avg = [float(sum(col))/len(col) for col in zip(*article_embedd)]
    return avg


In [3]:
# YELP product reviews dataset

import codecs

sentiment_dataset = codecs.open("../datasets/yelp-test.csv","r","utf-8").read().strip().split("\n")

print (sentiment_dataset[1])
print (" ")
print (sentiment_dataset[2])

"1","Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to fix a minor problem with a tire they put on. They \""fixed\"" it for free, and the very next morning I had the same issue. I called to complain, and the \""manager\"" didn't even apologize!!! So frustrated. Never going back.  They seem overpriced, too."
 
"2","Friendly staff, same starbucks fair you get anywhere else.  Sometimes the lines can get long."


In [4]:
# first, we define two folders, "corpus" - with the text and "labels", with the labels

corpus = []
labels = []

# be careful with this, the dataset is huge!
#for line in sentiment_dataset:
for line in sentiment_dataset[:1000]:
    text = line.split(",")[1].replace('"','')
    label = line.split(",")[0].replace('"','').replace("1","-1").replace("2","1")
    
    emb_text = text_embedding(text)
    if len(emb_text) > 0:
        corpus.append(emb_text)
        labels.append(label)
    
print ("done!")

done!


In [5]:
import numpy as np

# we use np array as they are more efficient
X = np.array(corpus)
y = np.array(labels)

In [6]:
#here's the documentation: http://scikit-learn.org/stable/supervised_learning.html#supervised-learning

from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

final_f1 = []
# we set that we do 10 fold cross validation
kf_total = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True)
# for each of the 10 round
for train, test in kf_total:
    # we define training and test embeddings and labels
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    
    # we train on the training set, using embeddings and labels
    classifier = GaussianNB().fit(X_train , y_train)
    
    # then we test it on the test set, we provide the embeddings and we make the classifier predict the labels
    y_pred = classifier.predict(X_test)
    
    # then we compare the prediction with the true test-labels using precision, recall and f1 (ignore the last None column)
    print (precision_recall_fscore_support(y_test, y_pred, average="macro"))
    f1_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[2]
    final_f1.append(f1_score)
    
print (" ")
print (sum(final_f1)/len(final_f1))

(0.5993431855500821, 0.5969551282051282, 0.595959595959596, None)
(0.6496598639455782, 0.6498397435897436, 0.6496847162446202, None)
(0.6001838235294117, 0.5873397435897436, 0.5689655172413794, None)
(0.5538555691554468, 0.5528846153846154, 0.5488721804511278, None)
(0.6939393939393939, 0.6923076923076923, 0.6897207486738064, None)
(0.5853858784893268, 0.5833333333333333, 0.5784825371336813, None)
(0.6764705882352942, 0.6764705882352942, 0.6764705882352942, None)
(0.6185185185185185, 0.6176470588235294, 0.6158088235294118, None)
(0.6201264488935722, 0.5931372549019608, 0.564344746162928, None)
(0.6273320895522387, 0.6115196078431373, 0.5954950235725511, None)
 
0.6083804477204398


/Users/federiconanni/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
#here's the documentation: http://scikit-learn.org/stable/supervised_learning.html#supervised-learning
from sklearn import svm

SVM = svm.SVC(kernel = "linear", C=1) 

final_f1 = []

kf_total = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True)

for train, test in kf_total:
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    classifier = SVM.fit(X_train , y_train)
    y_pred = classifier.predict(X_test)
    
    print (precision_recall_fscore_support(y_test, y_pred, average="macro"))
    f1_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[2]
    final_f1.append(f1_score)
print (" ")
print (sum(final_f1)/len(final_f1))

(0.7028985507246377, 0.7019230769230769, 0.6998799519807923, None)
(0.6696678671468588, 0.6698717948717949, 0.6697027324592133, None)
(0.700487012987013, 0.6979166666666667, 0.6980676328502415, None)
(0.6995192307692308, 0.6995192307692308, 0.6995192307692308, None)
(0.6193910256410255, 0.6193910256410255, 0.6193910256410255, None)
(0.6599025974025974, 0.6578525641025641, 0.6578099838969403, None)
(0.6156685808039377, 0.6151960784313726, 0.6151800327332242, None)
(0.666530612244898, 0.6666666666666666, 0.6665305705828315, None)
(0.6865793780687397, 0.6862745098039216, 0.686356668369954, None)
(0.5996677740863787, 0.5980392156862745, 0.5949263502454991, None)
 
0.6607364179528952


In [8]:
from sklearn.neighbors import KNeighborsClassifier

final_f1 = []
# we set that we do 10 fold cross validation
kf_total = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True)
# for each of the 10 round
for train, test in kf_total:
    # we define training and test embeddings and labels
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    
    # we train the classifier (using 50 neighbors, but you can change that)
    # we train on the training set, using embeddings and labels
    classifier = KNeighborsClassifier(n_neighbors=50).fit(X_train, y_train) 
    
    # then we test it on the test set, we provide the embeddings and we make the classifier predict the labels
    y_pred = classifier.predict(X_test)
    
    # then we compare the prediction with the true test-labels using precision, recall and f1 (ignore the last None column)
    print (precision_recall_fscore_support(y_test, y_pred, average="macro"))
    f1_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[2]
    final_f1.append(f1_score)
    
print (" ")
print (sum(final_f1)/len(final_f1))

(0.6552981552981553, 0.6450320512820513, 0.6419437340153453, None)
(0.7131837307152875, 0.6826923076923077, 0.6756982948007113, None)
(0.6338481338481339, 0.625, 0.6214833759590792, None)
(0.6416666666666666, 0.6362179487179487, 0.6347402597402596, None)
(0.6066176470588236, 0.592948717948718, 0.5833333333333334, None)
(0.5993431855500821, 0.5969551282051282, 0.595959595959596, None)
(0.637218045112782, 0.6341911764705883, 0.6333333333333334, None)
(0.5444444444444445, 0.5441176470588236, 0.5437788018433181, None)
(0.6066021867115223, 0.6035539215686274, 0.6019997938356871, None)
(0.6805084745762712, 0.6740196078431373, 0.6727272727272728, None)
 
0.6204997795547935


In [9]:
# homework 2 - nearest centroid

from sklearn.neighbors.nearest_centroid import NearestCentroid

final_f1 = []

kf_total = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True)
for train, test in kf_total:
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    
    classifier = NearestCentroid().fit(X_train, y_train) 
    
    y_pred = classifier.predict(X_test)
    
    print (precision_recall_fscore_support(y_test, y_pred, average="macro"))
    f1_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[2]
    final_f1.append(f1_score)
print (" ")
print (sum(final_f1)/len(final_f1))

(0.6292653552790044, 0.6290064102564102, 0.6290726817042607, None)
(0.5896358543417366, 0.5897435897435898, 0.5896306676008408, None)
(0.66, 0.6602564102564102, 0.6598639455782312, None)
(0.7011217948717949, 0.7011217948717949, 0.7, None)
(0.6696678671468588, 0.6698717948717949, 0.6697027324592133, None)
(0.66, 0.6602564102564102, 0.6598639455782312, None)
(0.646326530612245, 0.6464460784313726, 0.6463203021333062, None)
(0.6562756357670221, 0.6556372549019608, 0.6556873977086743, None)
(0.5650572831423895, 0.5649509803921569, 0.5649463464486459, None)
(0.615909090909091, 0.6145833333333333, 0.6142329778506973, None)
 
0.6389320997062101
